# ToDo
* Referencing the original model
* How to initialize a parameter calling a function in an abstract model? Explain, that it works the same way
* Objective subclass of Constraint?

# Motivation
The Pyomo-Documentation contains a [gallery](https://github.com/Pyomo/PyomoGallery/wiki), where the most popular introduction model to [GAMS](http://www.gams.com/) is rewritten using the Pyomo-package: [TRNSPRT](http://nbviewer.jupyter.org/github/Pyomo/PyomoGallery/blob/master/transport/transport.ipynb). 

However, in the pyomo gallery the model is created as a concrete model. This means, all sets and parameters need to be defined in the model file. For most practical applications it is desirable to create an abstract model that can be initialized with varying data, in order to use the model for more purposes. I'm going to show how to create the transport model as an abstract model, initialize it with the data of the standard example and solve the problem. I suggest you look through the example in the pyomo gallery before reading this post, because I won't repeat the explanation of some interesting details.

# The transport model
The transport model is an optimization of the transport costs of one good. Production capacities of the plants must not be exceeded, while the demand at the markets must be met. The transport costs are linear with respect to the freight and the distance between a plant and a market. For a description of the transport model using mathematical terms see the example in the [pyomo gallery](http://nbviewer.jupyter.org/github/Pyomo/PyomoGallery/blob/master/transport/transport.ipynb).

In the original description of the model, as well as in the replication in the pyomo gallery, simple characters are used as designators for sets, parameters and variables. While this might the mathematical convention, I'm going to deviate from that approach and use descriptive names.

# Implementation of the abstract model
In contrast to the example in the pyomo gallery, the model must be an object of the class `ConcreteModel`. 

In [1]:
from pyomo.environ import *
 
model = AbstractModel()

## Set definitions

In [2]:
model.plants = Set(doc="Canning plants")
model.markets = Set(doc="Markets")

## Parameter definitions

In [3]:
model.capacity_plant = Param(model.plants, doc='Capacity of a plant in cases')
model.demand_market = Param(model.markets, doc='Demand of a market in cases')
model.distance_in_thousand_miles = Param(model.plants, model.markets, doc='Distance between plant and marekt in thousands of miles')
model.freight_costs_per_case_and_thousand_miles = Param(initialize=90, doc='Freight in dollars per case per thousand miles')

In [4]:
def c_init(model, plant, market):
  return model.freight_costs_per_case_and_thousand_miles * model.distance_in_thousand_miles[plant,market] / 1000
model.freight_costs_per_case_in_thousands = Param(model.plants, model.markets, initialize=c_init, doc='Transport cost in thousands of dollar per case')

## Variable definitions

In [5]:
model.shipment_quantities_in_cases = Var(model.plants, model.markets, bounds=(0, None), doc='Shipment quantities in case')

## Constraints
* Here you can see how using descriptive names enhances the model readibility
* Explain how the supply_rule function is called

In [6]:
def supply_rule(model, plant):
  return sum(model.shipment_quantities_in_cases[plant,market] for market in model.markets) <= model.capacity_plant[plant]

def demand_rule(model, market):
  return sum(model.shipment_quantities_in_cases[plant, market] for plant in model.plants) >= model.demand_market[market] 

model.supply_constraint = Constraint(model.plants, rule=supply_rule, doc='Limit supply limit at each plant')
model.demand_constraint = Constraint(model.markets, rule=demand_rule, doc='Satisfy demand at each market')

## Objective

In [7]:
def objective_rule(model):
  return sum(model.freight_costs_per_case_in_thousands[plant,market]*model.shipment_quantities_in_cases[plant,market] 
             for plant in model.plants for market in model.markets)
model.objective = Objective(rule=objective_rule, sense=minimize, doc='Define objective function')

# Create a concrete instance of the abstract model

In [8]:
instance = model.create_instance(filename='trnsprt.dat')

## Retrieving the output

In [9]:
def pyomo_postprocess(options=None, instance=None, results=None):
  instance.shipment_quantities_in_cases.display()

## Running the model

In [10]:
from pyomo.opt import SolverFactory
opt = SolverFactory("glpk")
results = opt.solve(instance)
#sends results to stdout
results.write()
print("\nDisplaying Solution\n" + '-'*60)
pyomo_postprocess(None, instance, results)

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 153.675
  Upper bound: 153.675
  Number of objectives: 1
  Number of constraints: 6
  Number of variables: 7
  Number of nonzeros: 13
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.020014286041259766
# ----------------------------------------------------------
#   Solution Information
# -------------------------------